<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/05_Backtesting_com_Pyfolio_%7C_Sharpe%2C_Beta%2C_Volatilidade%2C_CAGR%2C_Drawdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web

!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import seaborn as sns

In [ ]:
!pip install pyfolio
import pyfolio as pf
import warnings
warnings.filterwarnings('ignore')

2. Obtendo e tratando os dados

In [ ]:
tickers = ["ABEV3.SA", "ITSA4.SA", "WEGE3.SA","USIM5.SA", "VALE3.SA", "^BVSP"]
dados_yahoo = web.get_data_yahoo(tickers, period="1y")["Adj Close"]
#dados_yahoo = web.get_data_yahoo(tickers, start="2017-01-01",end="2020-01-01")["Adj Close"]

[*********************100%***********************]  6 of 6 completed


In [ ]:
dados_yahoo

,ABEV3.SA,ITSA4.SA,USIM5.SA,VALE3.SA,WEGE3.SA,^BVSP
Date,,,,,,
2020-08-13,12.808670,9.532032,8.637742,54.357815,34.071579,100461.0
2020-08-14,12.857151,9.599703,8.727201,55.258522,34.205101,101353.0
2020-08-17,12.459607,9.309683,8.796781,56.051147,33.611691,99595.0
2020-08-18,12.682619,9.493746,9.393171,56.798733,33.478172,102065.0
2020-08-19,12.459607,9.425933,9.591969,56.195263,32.578167,100854.0
...,...,...,...,...,...,...
2021-08-09,17.160000,11.560000,21.520000,109.010002,36.160000,123019.0
2021-08-10,16.709999,11.470000,22.100000,110.059998,36.090000,122202.0
2021-08-11,16.570000,11.640000,22.030001,109.269997,36.150002,122056.0


In [ ]:
retorno = dados_yahoo.pct_change()
retorno 


,ABEV3.SA,ITSA4.SA,USIM5.SA,VALE3.SA,WEGE3.SA,^BVSP
Date,,,,,,
2020-08-13,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-14,0.003785,0.007099,0.010357,0.016570,0.003919,0.008879
2020-08-17,-0.030920,-0.030211,0.007973,0.014344,-0.017349,-0.017345
2020-08-18,0.017899,0.019771,0.067796,0.013338,-0.003972,0.024800
2020-08-19,-0.017584,-0.007143,0.021164,-0.010625,-0.026883,-0.011865
...,...,...,...,...,...,...
2021-08-09,0.003509,0.004344,-0.003242,-0.006290,-0.002208,0.001702
2021-08-10,-0.026224,-0.007785,0.026952,0.009632,-0.001936,-0.006641
2021-08-11,-0.008378,0.014821,-0.003167,-0.007178,0.001663,-0.001195


In [ ]:
retorno_acumulado = (1+retorno).cumprod()
retorno_acumulado.iloc[0]=1
retorno_acumulado

,ABEV3.SA,ITSA4.SA,USIM5.SA,VALE3.SA,WEGE3.SA,^BVSP
Date,,,,,,
2020-08-13,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2020-08-14,1.003785,1.007099,1.010357,1.016570,1.003919,1.008879
2020-08-17,0.972748,0.976673,1.018412,1.031152,0.986502,0.991380
2020-08-18,0.990159,0.995983,1.087457,1.044905,0.982584,1.015966
2020-08-19,0.972748,0.988869,1.110472,1.033803,0.956168,1.003912
...,...,...,...,...,...,...
2021-08-09,1.339718,1.212753,2.491392,2.005415,1.061295,1.224545
2021-08-10,1.304585,1.203311,2.558539,2.024732,1.059241,1.216412
2021-08-11,1.293655,1.221146,2.550435,2.010198,1.061002,1.214959


In [ ]:
carteira = 1000 *retorno_acumulado.iloc[:,:5]
carteira["saldo"]=carteira.sum(axis=1)
carteira["retorno"]=carteira["saldo"].pct_change()
carteira

,ABEV3.SA,ITSA4.SA,USIM5.SA,VALE3.SA,WEGE3.SA,saldo,retorno
Date,,,,,,,
2020-08-13,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,5000.000000,NaN
2020-08-14,1003.785013,1007.099307,1010.356696,1016.569968,1003.918868,5041.729852,0.008346
2020-08-17,972.747919,976.673477,1018.412051,1031.151596,986.502285,4985.487327,-0.011155
2020-08-18,990.158935,995.983416,1087.456799,1044.904638,982.583529,5101.087316,0.023187
2020-08-19,972.747919,988.869201,1110.471752,1033.802833,956.168394,5062.060099,-0.007651
...,...,...,...,...,...,...,...
2021-08-09,1339.717534,1212.753000,2491.391888,2005.415460,1061.295101,8110.572983,-0.001627
2021-08-10,1304.585021,1203.311136,2558.539057,2024.731825,1059.240611,8150.407651,0.004911
2021-08-11,1293.654973,1221.145746,2550.435123,2010.198480,1061.001650,8136.435971,-0.001714


3. Resultados

In [ ]:
pf.create_full_tear_sheet(carteira["retorno"], benchmark_rets = retorno["^BVSP"])

Start date,2020-08-13
End date,2021-08-13
Total months,11
,Backtest
Annual return,61.8%
Cumulative returns,60.0%
Annual volatility,24.7%
Sharpe ratio,2.08
Calmar ratio,4.78
Stability,0.90
Max drawdown,-12.9%


AttributeError: ignored